In [35]:
import os

from PIL import Image
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm, tqdm_notebook
import cv2

tf.logging.set_verbosity(tf.logging.INFO)

# load data

In [36]:
train_df = pd.read_csv('../input/train.csv')

In [37]:
train_df.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [38]:
X_tr = []
Y_tr = []
imges = train_df['id'].values
train_dir = '../input/train/train/'

In [39]:
temp = cv2.imread('../input/train.zip/0004be2cfeaba1c0361d39e2b000257b.jpg')
open('../input/train/train/0004be2cfeaba1c0361d39e2b000257b.jpg')

<_io.TextIOWrapper name='../input/train/train/0004be2cfeaba1c0361d39e2b000257b.jpg' mode='r' encoding='UTF-8'>

In [40]:
#Image Augment 
for img_id in imges:
    image = np.array(cv2.imread(train_dir + img_id))
    
    
    label = train_df[train_df['id'] == img_id]['has_cactus'].values[0]

    X_tr.append(image)
    Y_tr.append(label)  

    X_tr.append(np.flip(image))
    Y_tr.append(label)  

    X_tr.append(np.flipud(image))
    Y_tr.append(label)  

    X_tr.append(np.fliplr(image))
    Y_tr.append(label)  
                
X_tr = np.asarray(X_tr).astype('float32')/225

Y_tr = np.asarray(Y_tr)

In [41]:
X_tr.shape,Y_tr.shape

((70000, 32, 32, 3), (70000,))

In [42]:
from sklearn.model_selection import train_test_split

train_data, eval_data, train_labels, eval_labels = train_test_split(X_tr, Y_tr, test_size=0.3)

# load data test

In [43]:
# Load training and eval data
'''
((train_data, train_labels), (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required
'''

'\n((train_data, train_labels), (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()\n\ntrain_data = train_data/np.float32(255)\ntrain_labels = train_labels.astype(np.int32)  # not required\n\neval_data = eval_data/np.float32(255)\neval_labels = eval_labels.astype(np.int32)  # not required\n'

# build model

In [44]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    n = pool2.shape
    pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=2)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        
        accuracy = tf.metrics.accuracy(labels=labels, predictions=predictions["classes"], name='acc_op')
        metrics = {'accuracy': accuracy}
        tf.summary.scalar('accuracy', accuracy[1])
        
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"], name ='acc_val')
        }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [45]:
# Create the Estimator
cactus_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="./tmp/cactus_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tmp/cactus_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6aa1afd9b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [46]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [47]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

In [48]:
for i in range(10):
    cactus_classifier.train(input_fn=train_input_fn, steps=100)

    eval_results = cactus_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/cactus_convnet_model/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1000 into ./tmp/cactus_convnet_model/model.ckpt.
INFO:tensorflow:loss = 0.48714378, step = 1000
INFO:tensorflow:Saving checkpoints for 1100 into ./tmp/cactus_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.4618557.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-13T03:08:41Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/cactus_convnet_model/model.ckpt-1100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-13-03:08:42
INFO:tenso

INFO:tensorflow:Loss for final step: 0.30235493.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-13T03:08:55Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/cactus_convnet_model/model.ckpt-1700
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-13-03:08:56
INFO:tensorflow:Saving dict for global step 1700: accuracy = 0.8182857, global_step = 1700, loss = 0.34218705
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1700: ./tmp/cactus_convnet_model/model.ckpt-1700
{'accuracy': 0.8182857, 'loss': 0.34218705, 'global_step': 1700}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/cactus_convnet_model/model.ckpt-1700
INFO:tensorflow:Running lo

In [49]:
#!tensorboard --logdir=./tmp/cactus_convnet_model

# predict

In [50]:
from os import listdir
from os.path import isfile, join

mypath = '../input/test/test'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [51]:
pred = []

for img in files:
    image = np.array(cv2.imread(mypath + '/' + img))
    pred.append(image)

pred = np.asarray(pred).astype('float32')/225

In [52]:
pred.shape

(4000, 32, 32, 3)

In [53]:
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": pred},
    num_epochs=1,
    shuffle=False)

In [54]:
result = cactus_classifier.predict(input_fn = pred_input_fn)

In [55]:
temp = [x['probabilities'][1] for x in result]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/cactus_convnet_model/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [56]:
pd.DataFrame({'id': files, 'has_cactus':temp}).to_csv('submission.csv', index = False)